# markdown.obsidian.links
> Functions for parsing internal links in [Obsidian.md](https://obsidian.md/) style markdown.

Obsidian uses both Markdown style links and Wikilinks as [internal links](https://help.obsidian.md/How+to/Internal+link). Markdown style links are of the form `[text_shown](link)` whereas Wikilinks are of the form `[[link_to_markdown#possible_anchor_to_header|text_shown]]`. They have an exclamation mark `!` if they are embedded.

In [ ]:
#| default_exp markdown.obsidian.links

In [ ]:
#| export
from __future__ import annotations
from deprecated import deprecated
from enum import Enum
from pathlib import Path
import re
from trouver.helper import (
    find_regex_in_text, text_from_file
)
from trouver.markdown.obsidian.vault import (
    all_paths_to_notes_in_vault, VaultNote, NoteDoesNotExistError
)
from typing import Union


In [ ]:
#| export
# TODO Make it so that these patterns don't capture latex code
INTERNAL_LINK_PATTERN = r'!?\[\[.*?\]\]' 
WIKILINK_PATTERN = r'!?\[\[[^\]]+\]\]'
EMBEDDED_WIKILINK_PATTERN = r'!\[\[[^\]]+\]\]'
# WIKILINK_CAPTURE = r'!?\[\[([^#\]\|]+)(#[^\]\|]+)?(\|[^\]]+)?\]\]'
# Note that MARKDOWNLINK_PATTERN captures whitespace characters in its link, even though Obsidian
# does not. This is implmeneted to find if any misformats in the Obsidian Markdown files.
MARKDOWNLINK_PATTERN = r'!?\[[^\]]+\]\([^)]+\)'  
EMBEDDED_MARKDOWNLINK_PATERN = r'!\[[^\]]+\]\([^)]+\)'
EMBEDDED_PATTERN = f'{EMBEDDED_WIKILINK_PATTERN}|{EMBEDDED_MARKDOWNLINK_PATERN}'
# MARKDOWNLINK_CAPTURE = r'!?\[([^\]]+)\]\(([^)#])+(#[^)]+)?\)'

In [ ]:
from os import PathLike
from fastcore.test import *

## Finding links in text via indices

In [ ]:
#| export
def find_links_in_markdown_text(
        text: str
        ) -> list[tuple]: # Each tuple is of the form `(a,b)` where `text[a:b]` is an obsidian internal link.
    """Returns ranges in the markdown text string
    where internal links occur.

    # TODO: rename this function, say to link_ranges_in_text, 
    # because it is confusing when there is a links_from_text function below.

    **See Also**

    - `links_from_text`
    """
    regex = f'{WIKILINK_PATTERN}|{MARKDOWNLINK_PATTERN}'
    return find_regex_in_text(text, pattern=regex)


`find_links_in_markdown_text` returns a list of indices in a string in which the links are located.

In [ ]:
# TODO: add markdown links to example

In [ ]:
tutorial_text = r'''
This is an Obsidian note. It has some [[this_is_the_note_to_which_the_link_points|links]]!
Links are pretty neat. They can [[this_text_is_not_actually_shown|connect notes]] for you.
The following will create a link to the note `some_note`; the displayed text is `some_note`: [[some_note]]
You can also embed the contents of one note into another note. ![[note_being_embedded]].
The contents of `note_being_embedded` will be displayed when you view the note in Obsidian's view mode.
You can make anchors in links. For example [[note#This is a header title]] is a link to the note named
`note` and more specifically to the theader with title `This is a header title`.

The above links are all Wikilinks. Obsidian also supports Markdownlinks, e.g. [This is the text shown](This is the link.)

If the note of a link does not exist in an Obsidian vault, then Obsidian will create the note.
Even if the note does not have a header with title specified by the anchor of a link, Obsidian
will still open the note; it will not go to any particular header, however.
'''

ranges = find_links_in_markdown_text(tutorial_text)
match_strs = [tutorial_text[start:end] for start, end in ranges]
test_eq(match_strs, [
    '[[this_is_the_note_to_which_the_link_points|links]]', 
    '[[this_text_is_not_actually_shown|connect notes]]',
    '[[some_note]]',
    '![[note_being_embedded]]',
    '[[note#This is a header title]]',
    '[This is the text shown](This is the link.)'])

### Longer Example

The following example is of a note whose contents are based on an excerpt from Vakil's *The Rising Sea - Foundations of Algebraic Geometry*.

In [ ]:
text = r'''
---
cssclass: clean-embeds
aliases: [foag_relative_cotangent_sheaf, foag_relative_tangent_sheaf]
tags: [_meta/definition, _meta/notation, _meta/literature_note, _reference/vakil_rising_sea, relative_cotangent_sheaf, relative_tangent_sheaf, diagonal_map/scheme, conormal_sheaf, dual/coherent_sheaf, morphism/schemes]
---
# Global definition[^1]
Let $\pi: X \rightarrow Y$ be a morphism of schemes. Define the **relative cotangent sheaf** $\Omega_{X/Y}$ or $\Omega_{\pi}$ as the [[foag_conormal_sheaf_of_a_locally_closed_embedding#For a locally closed embedding 2 4|conormal sheaf]] $\mathscr{N}^\vee_{X/X \times_Y X}$ of the diagonal[^2].

# Relative tangent sheaf
Define the **relative tangent sheaf** $\mathscr{T}_{X/Y}$ as the [[foag_dual_sheaf|dual]] $\mathcal{Hom}(\Omega_{X/Y}, \mathscr{O}_X)$[^3]

[^3]: ![[foag_notation_Hom_sheaf_hom]]

# Other
We now define $\mathrm{d}: \hat{\partial}_{\mathrm{X}} \rightarrow \Omega_{\mathrm{X} / \mathrm{Y}}$. Let $\operatorname{pr}_{1}: X \times_{\mathrm{Y}} X \rightarrow X$ and $\operatorname{pr}_{2}: X \times_{\mathrm{Y}} X \rightarrow X$ be the two projections. Then define $\mathrm{d}: \mathscr{O}_{\mathrm{X}} \rightarrow \Omega_{\mathrm{X} / \mathrm{Y}}$ on the open set $\mathrm{U}$ as follows:
$$
d f=p r_{2}^{*} f-p r_{1}^{*} f
$$
(Warning: this is not a morphism of quasicoherent sheaves on $X$, although it $i$ s $\mathscr{O}_{\mathrm{Y}}$-linear in the only possible meaning of that phrase.) We will soon see that $\mathrm{d}$ is indeed a derivation of the sheaf $\mathscr{O}_{\mathrm{X}}$ (in the only possible meaning of the phrase), and at the same time see that our new notion of differentials agrees with our old definition on affine open sets, and hence globalizes the definition. Note that for any open subset $U \subset X, d$ induces a map
$$
\Gamma\left(\mathrm{U}, \mathscr{O}_{\mathrm{X}}\right) \rightarrow \Gamma\left(\mathrm{U}, \Omega_{\mathrm{X} / \mathrm{Y}}\right)
$$
which we also call d, and interpret as "taking the derivative".

# See Also
- [[foag_notation_T_X_Y_relative_tangent_sheaf]]
- [[foag_notation_Omega_X_Y_relative_cotangent_sheaf]]
- [[foag_21.2.Q]]
- [[foag_sheaf_of_relative_i_forms]]

# Meta
## References
![[_reference_foag]]

## Citations and Footnotes
[^1]: Vakil, 21.2.20, Page 572
[^2]: Note that the diagonal morphism is a locally closed embedding.
'''

ranges = find_links_in_markdown_text(text)
for match_range in ranges:
    print(text[match_range[0]:match_range[1]])

[[foag_conormal_sheaf_of_a_locally_closed_embedding#For a locally closed embedding 2 4|conormal sheaf]]
[[foag_dual_sheaf|dual]]
![[foag_notation_Hom_sheaf_hom]]
[[foag_notation_T_X_Y_relative_tangent_sheaf]]
[[foag_notation_Omega_X_Y_relative_cotangent_sheaf]]
[[foag_21.2.Q]]
[[foag_sheaf_of_relative_i_forms]]
![[_reference_foag]]
